# Google Sheet 연동 및 변수 병합

In [ ]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/drive/MyDrive/valid-might-460212-k2-aabca356a049.json'

import tempfile
from google.cloud import storage
import pandas as pd

GCS_BUCKET = 'de-project2-bucket-1'
GCS_CSV_PREFIX = 'kyobo/csv'
GCS_mood = 'kyobo/csv/combined/inference_mood_batches/mood_classification.csv'

client = storage.Client()
bucket = client.bucket(GCS_BUCKET)

# Colab에서 GCS 파일을 로컬로 다운로드
# !gsutil cp gs://de-project2-bucket-1/kyobo/csv/combined/after_preprocessing_novel.csv /content/drive/MyDrive/code/after_preprocessing_novel.csv
# !gsutil cp gs://de-project2-bucket-1/kyobo/csv/combined/inference_mood_batches/mood_classification.csv /content/drive/MyDrive/code/mood_novel.csv
# !gsutil cp gs://de-project2-bucket-1/kyobo/csv/combined/novel-image_classification.csv /content/drive/MyDrive/code/layout_novel.csv
!gsutil cp gs://de-project2-bucket-1/kyobo/csv/combined/object_detection_novel_20250614_MJ.csv /content/drive/MyDrive/code/object_detection_novel.csv
# !gsutil cp gs://de-project2-bucket-1/kyobo/csv/ocr_results_aggregated.csv /content/drive/MyDrive/code/ocr_novel.csv

Copying gs://de-project2-bucket-1/kyobo/csv/combined/object_detection_novel_20250614_MJ.csv...
\
Operation completed over 1 objects/1.1 MiB.                                      


# novel_df

In [ ]:
# from airflow.providers.google.cloud.hooks.gcs import GCSHook
# import pandas as pd

# gcs_hook = GCSHook(gcp_conn_id='google_cloud_default')
# bucket_name = 'de-project2-bucket-1'

# def read_csv_from_gcs():
#     object_name = 'kyobo/today/today_metadata.csv'
#     local_path = '/tmp/today_metadata.csv'
#     gcs_hook.download(bucket_name=bucket_name, object_name=object_name, filename=local_path)
#     novel_df = pd.read_csv(local_path, encoding='utf-8-sig')
#     novel_df['category_code'] = novel_df['image_url'].str.extract(r'images/([^/]+)/')
#     print(novel_df.head())
#     return novel_df

In [ ]:
import pandas as pd
novel_df = pd.read_csv('/content/drive/MyDrive/code/after_preprocessing_novel.csv', encoding='utf-8-sig')
novel_df['category_code'] = novel_df['image_url'].str.extract(r'images/([^/]+)/')
novel_df.head()

,index,product_code,product_id,title,list_price,sale_price,discount_rate,point_rate,expected_points,author,publisher,pub_date,category,image_url,category_code
0,1,9.788998e+12,S000001632467,모순,"13,000","11,700",10%,5%,650,양귀자,쓰다,20130401.0,소설,https://storage.googleapis.com/de-project2-buc...,010101
1,2,9.791162e+12,S000001808685,파과,"15,800","14,220",10%,5%,790,구병모,위즈덤하우스,20180416.0,소설,https://storage.googleapis.com/de-project2-buc...,010101
2,3,9.788936e+12,S000000610612,소년이 온다,"15,000","13,500",10%,5%,750,한강,창비,20140519.0,소설,https://storage.googleapis.com/de-project2-buc...,010101
3,4,9.788958e+12,S000216345011,포기할 자유,"18,500","16,650",10%,5%,920,이재구,아마존북스,20250421.0,소설,https://storage.googleapis.com/de-project2-buc...,010101
4,5,9.788936e+12,S000216111714,혼모노,"18,000","16,200",10%,5%,900,성해나,창비,20250328.0,소설,https://storage.googleapis.com/de-project2-buc...,010101


In [ ]:
file_path = '/content/drive/MyDrive/code/category_code_file.csv'
category_code_file = pd.read_csv(file_path)
category_code_file['category_code'] = category_code_file['category_code'].astype(str).str.zfill(6)
category_code_file.head()

,TopCategory,MidCategory,SubCategory,LeafCategory,category_code,mid_code,sub_code,leaf_code
0,국내도서,소설,한국소설,한국소설일반,010101,1,1,1
1,국내도서,소설,한국소설,고전소설/문학선,010102,1,1,2
2,국내도서,소설,한국소설,공포/호러소설,010103,1,1,3
3,국내도서,소설,한국소설,미스터리/스릴러소설,010104,1,1,4
4,국내도서,소설,한국소설,역사/대하소설,010105,1,1,5


In [ ]:
novel_df = pd.merge(
    novel_df,
    category_code_file,
    on='category_code',
    how='left'
)
novel_df.head()

,index,product_code,product_id,title,list_price,sale_price,discount_rate,point_rate,expected_points,author,...,category,image_url,category_code,TopCategory,MidCategory,SubCategory,LeafCategory,mid_code,sub_code,leaf_code
0,1,9.788998e+12,S000001632467,모순,"13,000","11,700",10%,5%,650,양귀자,...,소설,https://storage.googleapis.com/de-project2-buc...,010101,국내도서,소설,한국소설,한국소설일반,1.0,1.0,1.0
1,2,9.791162e+12,S000001808685,파과,"15,800","14,220",10%,5%,790,구병모,...,소설,https://storage.googleapis.com/de-project2-buc...,010101,국내도서,소설,한국소설,한국소설일반,1.0,1.0,1.0
2,3,9.788936e+12,S000000610612,소년이 온다,"15,000","13,500",10%,5%,750,한강,...,소설,https://storage.googleapis.com/de-project2-buc...,010101,국내도서,소설,한국소설,한국소설일반,1.0,1.0,1.0
3,4,9.788958e+12,S000216345011,포기할 자유,"18,500","16,650",10%,5%,920,이재구,...,소설,https://storage.googleapis.com/de-project2-buc...,010101,국내도서,소설,한국소설,한국소설일반,1.0,1.0,1.0
4,5,9.788936e+12,S000216111714,혼모노,"18,000","16,200",10%,5%,900,성해나,...,소설,https://storage.googleapis.com/de-project2-buc...,010101,국내도서,소설,한국소설,한국소설일반,1.0,1.0,1.0


In [ ]:
unmatched_df = novel_df[novel_df['TopCategory'].isna()]
unmatched_codes = unmatched_df['category_code'].dropna().unique()
print(unmatched_codes)

['3813' '05170113' '3817' '052307' '05170503' '05131925' '05170307'
 '17110101' '17110107' '150509' '051915' '170103' '051521' '051911'
 '130107' '052117' '29051903' '052011' '05131927' '05131931' '051519'
 '3833' '052305' '052121' '05131301' '051303' '170921' '051523' '051525'
 '052103' '051503' '05250301' '290501' '170507' '05131311' '05132101'
 '05131321' '05131307' '17090901' '17110307' '05131703' '17090923'
 '051311' '150105' '17010101' '17091301' '05131304' '05131707' '17010707'
 '17010705' '17090921' '05170121' '05131511' '051301' '131913' '05250111'
 '05131919' '150111' '05030503' '05250303' '05290117' '17090701'
 '17090917' '17010111' '051509' '05131303' '05170301' '05050301' '050901'
 '050907' '051101' '051309' '051517' '051529' '05170109' '05170111'
 '05170305' '05170501' '05170505' '05170507' '05170515' '05170517'
 '051903' '052119' '052123' '05250103' '05290305' '130105' '13230325'
 '13230501' '13230717' '150304' '150323' '17010105' '17091102' '17110117'
 '290503' '2905110

In [ ]:
# unmatched_codes 삭제
novel_df = novel_df[~novel_df['category_code'].isin(unmatched_codes)]

In [ ]:
# 자료형 올바르게 변환
cols_to_convert = ['product_code', 'mid_code', 'sub_code', 'leaf_code']
novel_df[cols_to_convert] = novel_df[cols_to_convert].astype('Int64').astype(str)

# 날짜형 변환
novel_df['pub_date'] = pd.to_datetime(novel_df['pub_date'].astype('Int64').astype(str), format='%Y%m%d', errors='coerce')
novel_df['pub_date']

,pub_date
0,2013-04-01
1,2018-04-16
2,2014-05-19
3,2025-04-21
4,2025-03-28
...,...
28196,2023-04-30
28197,2023-10-03
28198,2015-09-25
28199,2013-11-01


In [ ]:
novel_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28070 entries, 0 to 28200
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   index            28070 non-null  int64         
 1   product_code     28070 non-null  object        
 2   product_id       28070 non-null  object        
 3   title            28070 non-null  object        
 4   list_price       28070 non-null  object        
 5   sale_price       28070 non-null  object        
 6   discount_rate    28070 non-null  object        
 7   point_rate       28070 non-null  object        
 8   expected_points  28070 non-null  object        
 9   author           28070 non-null  object        
 10  publisher        28070 non-null  object        
 11  pub_date         28070 non-null  datetime64[ns]
 12  category         28070 non-null  object        
 13  image_url        28070 non-null  object        
 14  category_code    28070 non-null  object    



---


# mood label

In [ ]:
import pandas as pd
mood_df = pd.read_csv('/content/drive/MyDrive/code/mood_novel.csv', encoding='utf-8-sig')
mood_df = mood_df.rename(columns={'mood': 'mood_label'})
mood_df.head()

,index,product_id,mood_label
0,13,S000216110908,neutral
1,7,S000216176410,positive_high
2,9,S000000610650,positive_low
3,15,S000000778206,negative_low
4,14,S000215893009,negative_low


In [ ]:
# product_id 기준으로 left join (novel_df 기준)
novel_df = novel_df.merge(mood_df[['product_id', 'mood_label']], on='product_id', how='left')

In [ ]:
novel_df.head()

,index,product_code,product_id,title,list_price,sale_price,discount_rate,point_rate,expected_points,author,...,image_url,category_code,TopCategory,MidCategory,SubCategory,LeafCategory,mid_code,sub_code,leaf_code,mood_label
0,1,9788998441012,S000001632467,모순,"13,000","11,700",10%,5%,650,양귀자,...,https://storage.googleapis.com/de-project2-buc...,010101,국내도서,소설,한국소설,한국소설일반,1,1,1,neutral
1,2,9791162203620,S000001808685,파과,"15,800","14,220",10%,5%,790,구병모,...,https://storage.googleapis.com/de-project2-buc...,010101,국내도서,소설,한국소설,한국소설일반,1,1,1,positive_high
2,3,9788936434120,S000000610612,소년이 온다,"15,000","13,500",10%,5%,750,한강,...,https://storage.googleapis.com/de-project2-buc...,010101,국내도서,소설,한국소설,한국소설일반,1,1,1,negative_high
3,4,9788957753354,S000216345011,포기할 자유,"18,500","16,650",10%,5%,920,이재구,...,https://storage.googleapis.com/de-project2-buc...,010101,국내도서,소설,한국소설,한국소설일반,1,1,1,negative_low
4,5,9788936439743,S000216111714,혼모노,"18,000","16,200",10%,5%,900,성해나,...,https://storage.googleapis.com/de-project2-buc...,010101,국내도서,소설,한국소설,한국소설일반,1,1,1,negative_high


In [ ]:
novel_df["mood_label"].unique()

array(['neutral', 'positive_high', 'negative_high', 'negative_low',
       'positive_low'], dtype=object)



---


# layout

In [ ]:
import pandas as pd
layout_df = pd.read_csv('/content/drive/MyDrive/code/layout_novel.csv', encoding='utf-8-sig')
layout_df.head()

,product_id,design_label
0,S000001632467,typographic
1,S000001808685,illustration
2,S000000610612,typographic
3,S000216345011,photo
4,S000216111714,photo


In [ ]:
# product_id 기준으로 left join (novel_df 기준)
novel_df = novel_df.merge(layout_df[['product_id', 'design_label']], on='product_id', how='left')
novel_df.head()

,index,product_code,product_id,title,list_price,sale_price,discount_rate,point_rate,expected_points,author,...,category_code,TopCategory,MidCategory,SubCategory,LeafCategory,mid_code,sub_code,leaf_code,mood_label,design_label
0,1,9788998441012,S000001632467,모순,"13,000","11,700",10%,5%,650,양귀자,...,010101,국내도서,소설,한국소설,한국소설일반,1,1,1,neutral,typographic
1,2,9791162203620,S000001808685,파과,"15,800","14,220",10%,5%,790,구병모,...,010101,국내도서,소설,한국소설,한국소설일반,1,1,1,positive_high,illustration
2,3,9788936434120,S000000610612,소년이 온다,"15,000","13,500",10%,5%,750,한강,...,010101,국내도서,소설,한국소설,한국소설일반,1,1,1,negative_high,typographic
3,4,9788957753354,S000216345011,포기할 자유,"18,500","16,650",10%,5%,920,이재구,...,010101,국내도서,소설,한국소설,한국소설일반,1,1,1,negative_low,photo
4,5,9788936439743,S000216111714,혼모노,"18,000","16,200",10%,5%,900,성해나,...,010101,국내도서,소설,한국소설,한국소설일반,1,1,1,negative_high,photo


In [ ]:
novel_df["design_label"].unique()

array(['typographic', 'illustration', 'photo'], dtype=object)

# Object Detection

In [ ]:
import pandas as pd
object_df = pd.read_csv('/content/drive/MyDrive/code/object_detection_novel.csv', encoding='utf-8-sig')
object_df.head()

,product_id,title_position,title_direction,detected_objects
0,S000001632467,하중,가로,animal
1,S000001808685,중중,알수없음,flower
2,S000000610612,중중,세로,flower
3,S000216345011,하중,가로,landscape
4,S000216111714,하오,가로,없음


In [ ]:
# product_id 기준으로 left join (novel_df 기준)
novel_df = novel_df.merge(object_df, on='product_id', how='left')
novel_df.head()

,index,product_code,product_id,title,list_price,sale_price,discount_rate,point_rate,expected_points,author,...,SubCategory,LeafCategory,mid_code,sub_code,leaf_code,mood_label,design_label,title_position,title_direction,detected_objects
0,1,9788998441012,S000001632467,모순,"13,000","11,700",10%,5%,650,양귀자,...,한국소설,한국소설일반,1,1,1,neutral,typographic,하중,가로,animal
1,2,9791162203620,S000001808685,파과,"15,800","14,220",10%,5%,790,구병모,...,한국소설,한국소설일반,1,1,1,positive_high,illustration,중중,알수없음,flower
2,3,9788936434120,S000000610612,소년이 온다,"15,000","13,500",10%,5%,750,한강,...,한국소설,한국소설일반,1,1,1,negative_high,typographic,중중,세로,flower
3,4,9788957753354,S000216345011,포기할 자유,"18,500","16,650",10%,5%,920,이재구,...,한국소설,한국소설일반,1,1,1,negative_low,photo,하중,가로,landscape
4,5,9788936439743,S000216111714,혼모노,"18,000","16,200",10%,5%,900,성해나,...,한국소설,한국소설일반,1,1,1,negative_high,photo,하오,가로,없음


In [ ]:
novel_df["title_position"].unique()

array(['하중', '중중', '하오', '상중', '중왼', '상왼', '상오', '중오', '감지안됨', '하왼'],
      dtype=object)

In [ ]:
novel_df["title_direction"].unique()

array(['가로', '알수없음', '세로', '감지안됨'], dtype=object)

In [ ]:
novel_df["detected_objects"].unique()

array(['animal', 'flower', 'landscape', '없음', 'person', 'river', 'plant',
       'cloud', 'house', 'planet', 'clock', 'building', 'palace', 'hand',
       'food', 'vehicle', 'weapon', 'flag', 'sea', 'chair', 'airplane',
       'electronic device', 'insect', 'drink', 'lightbulb', 'mirror',
       'helicopter', 'star', 'desk', 'instrument', 'pencil', 'castle'],
      dtype=object)

# OCR

In [ ]:
import pandas as pd
ocr_df = pd.read_csv('/content/drive/MyDrive/code/ocr_novel.csv', encoding='utf-8-sig')
ocr_df = ocr_df.rename(columns={
    '텍스트(리스트)': 'text_list',
    '언어(리스트)': 'lang_list',
    '텍스트 너비(리스트)': 'text_width_list',
    '텍스트 높이(리스트)': 'text_height_list',
    '이미지 내 면적 비율(리스트)': 'text_ratio_list'
})
ocr_df.head()

,판매상품ID,Image URL,text_list,lang_list,text_width_list,text_height_list,text_ratio_list
0,S000001632467,https://storage.googleapis.com/de-project2-buc...,"색, 내 상의 부미논 나무있다, 기사비 칸 안 심음 만한 깊이도 없다, 이자계 신아...","ko, ko, ko, ko","52, 108, 148, 114","84, 16, 14, 16","0.0329, 0.013, 0.0156, 0.0138"
1,S000001808685,https://storage.googleapis.com/de-project2-buc...,"구멍보 장편소선, 유지철, 영화 <파과> 원작 소설, 전 세계 13개국 수습, 뉴용...","ko, ko, ko, sw","84, 206, 244, 42","16, 24, 16, 12","0.0103, 0.0381, 0.0301, 0.0039"
2,S000000610612,https://storage.googleapis.com/de-project2-buc...,"#향, $","ko, unknown","65, 30","119, 58","0.0593, 0.0133"
3,S000216345011,https://storage.googleapis.com/de-project2-buc...,"물, 흙, 피보다이념 이념보다 톤음 쫓는, 핑제들의 비극적인 대서사시!, 가주뭄 비...","ko, ko, ko, ko, ko, ko","44, 14, 162, 150, 216, 32","172, 42, 20, 20, 14, 8","0.0568, 0.0044, 0.0243, 0.0225, 0.0227, 0.0019"
4,S000216111714,https://storage.googleapis.com/de-project2-buc...,"@모뇌, 지금 가장 성명리고, ""넷플리스 왜 보나, 뜨거오 이I, 성해나 책 보면 ...","ko, ko, ko, ko, ko, ko, ko, ko, ko","68, 70, 128, 44, 170, 42, 44, 38, 244","24, 12, 20, 12, 24, 14, 14, 12, 14","0.0124, 0.0064, 0.0194, 0.004, 0.0309, 0.0045,..."


In [ ]:
# 전체 텍스트 비율 계산
ocr_df['text_ratio_sum'] = ocr_df['text_ratio_list'].apply(
    lambda x: sum([float(i.strip()) for i in str(x).split(',') if i.strip() != '']) if pd.notnull(x) else 0
)
ocr_df[['판매상품ID', 'text_ratio_list', 'text_ratio_sum']].head()

,판매상품ID,text_ratio_list,text_ratio_sum
0,S000001632467,"0.0329, 0.013, 0.0156, 0.0138",0.0753
1,S000001808685,"0.0103, 0.0381, 0.0301, 0.0039",0.0824
2,S000000610612,"0.0593, 0.0133",0.0726
3,S000216345011,"0.0568, 0.0044, 0.0243, 0.0225, 0.0227, 0.0019",0.1326
4,S000216111714,"0.0124, 0.0064, 0.0194, 0.004, 0.0309, 0.0045,...",0.1117


In [ ]:
ocr_df = ocr_df.rename(columns={'판매상품ID': 'product_id'})
ocr_df = ocr_df.drop(columns=['Image URL'])
ocr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253302 entries, 0 to 253301
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   product_id        253302 non-null  object 
 1   text_list         251773 non-null  object 
 2   lang_list         251773 non-null  object 
 3   text_width_list   251773 non-null  object 
 4   text_height_list  251773 non-null  object 
 5   text_ratio_list   251773 non-null  object 
 6   text_ratio_sum    253302 non-null  float64
dtypes: float64(1), object(6)
memory usage: 13.5+ MB


In [ ]:
# product_id 기준으로 left join (novel_df 기준)
novel_df = novel_df.merge(ocr_df, on='product_id', how='left')
novel_df.head()

,index,product_code,product_id,title,list_price,sale_price,discount_rate,point_rate,expected_points,author,...,design_label,title_position,title_direction,detected_objects,text_list,lang_list,text_width_list,text_height_list,text_ratio_list,text_ratio_sum
0,1,9788998441012,S000001632467,모순,"13,000","11,700",10%,5%,650,양귀자,...,typographic,하중,가로,animal,"색, 내 상의 부미논 나무있다, 기사비 칸 안 심음 만한 깊이도 없다, 이자계 신아...","ko, ko, ko, ko","52, 108, 148, 114","84, 16, 14, 16","0.0329, 0.013, 0.0156, 0.0138",0.0753
1,2,9791162203620,S000001808685,파과,"15,800","14,220",10%,5%,790,구병모,...,illustration,중중,알수없음,flower,"구멍보 장편소선, 유지철, 영화 <파과> 원작 소설, 전 세계 13개국 수습, 뉴용...","ko, ko, ko, sw","84, 206, 244, 42","16, 24, 16, 12","0.0103, 0.0381, 0.0301, 0.0039",0.0824
2,3,9788936434120,S000000610612,소년이 온다,"15,000","13,500",10%,5%,750,한강,...,typographic,중중,세로,flower,"#향, $","ko, unknown","65, 30","119, 58","0.0593, 0.0133",0.0726
3,4,9788957753354,S000216345011,포기할 자유,"18,500","16,650",10%,5%,920,이재구,...,photo,하중,가로,landscape,"물, 흙, 피보다이념 이념보다 톤음 쫓는, 핑제들의 비극적인 대서사시!, 가주뭄 비...","ko, ko, ko, ko, ko, ko","44, 14, 162, 150, 216, 32","172, 42, 20, 20, 14, 8","0.0568, 0.0044, 0.0243, 0.0225, 0.0227, 0.0019",0.1326
4,5,9788936439743,S000216111714,혼모노,"18,000","16,200",10%,5%,900,성해나,...,photo,하오,가로,없음,"@모뇌, 지금 가장 성명리고, ""넷플리스 왜 보나, 뜨거오 이I, 성해나 책 보면 ...","ko, ko, ko, ko, ko, ko, ko, ko, ko","68, 70, 128, 44, 170, 42, 44, 38, 244","24, 12, 20, 12, 24, 14, 14, 12, 14","0.0124, 0.0064, 0.0194, 0.004, 0.0309, 0.0045,...",0.1117


# Color

In [ ]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/drive/MyDrive/valid-might-460212-k2-aabca356a049.json'

import tempfile
from google.cloud import storage
import pandas as pd

GCS_BUCKET = 'de-project2-bucket-1'
GCS_CSV_PREFIX = 'kyobo/csv'

client = storage.Client()
bucket = client.bucket(GCS_BUCKET)

# Colab에서 GCS 파일을 로컬로 다운로드
!gsutil cp gs://de-project2-bucket-1/kyobo/csv/combined/color_novel.csv /content/drive/MyDrive/code/color_novel.csv

Copying gs://de-project2-bucket-1/kyobo/csv/combined/color_novel.csv...
\ [1 files][  5.3 MiB/  5.3 MiB]                                                
Operation completed over 1 objects/5.3 MiB.                                      


In [ ]:
import pandas as pd

# cp949 또는 euc-kr 사용 시도
try:
    color_df = pd.read_csv('/content/drive/MyDrive/code/color_novel.csv', encoding='cp949')
except UnicodeDecodeError:
    color_df = pd.read_csv('/content/drive/MyDrive/code/color_novel.csv', encoding='euc-kr')

color_df.head()

<ipython-input-6-2663767699>:5: DtypeWarning: Columns (97,98,103,104,109,110,115,116) have mixed types. Specify dtype option on import or set low_memory=False.
  color_df = pd.read_csv('/content/drive/MyDrive/code/color_novel.csv', encoding='cp949')


,product_id,HEX1,color1,비율1,H1(각도),S1(%),L1(%),HEX2,color2,비율2,...,비율19,H19(각도),S19(%),L19(%),HEX20,color20,비율20,H20(각도),S20(%),L20(%)
0,S000001632467,#f9f8f2,pastel_yellow,87.8,51.43,36.84,96.27,#000000,검정색 계열,10.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,S000001808685,#b57894,basic_purple,99.7,332.46,29.19,59.02,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,S000000610612,#4f4d38,basic_yellow,85.7,54.78,17.04,26.47,#e27f05,basic_orange,13.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,S000216345011,#dad9d9,흰색 계열,99.5,0.00,1.33,85.29,#e3df84,pastel_yellow,0.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,S000216111714,#88917a,basic_green,79.5,83.48,9.47,52.35,#8d3019,basic_red,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import pandas as pd

# 결측치 개수 계산
missing_counts = color_df.isnull().sum()

# 출력 옵션 설정 (출력 제한 해제)
pd.set_option('display.max_rows', None)  # 모든 행 출력 허용

print(missing_counts)

product_id        0
HEX1              5
color1            5
비율1               5
H1(각도)            5
S1(%)             5
L1(%)             5
HEX2          18368
color2        18368
비율2           18368
H2(각도)        18368
S2(%)         18368
L2(%)         18368
HEX3          23189
color3        23189
비율3           23189
H3(각도)        23189
S3(%)         23189
L3(%)         23189
HEX4          25563
color4        25563
비율4           25563
H4(각도)        25563
S4(%)         25563
L4(%)         25563
HEX5          26757
color5        26757
비율5           26757
H5(각도)        26757
S5(%)         26757
L5(%)         26757
HEX6          27423
color6        27423
비율6           27423
H6(각도)        27423
S6(%)         27423
L6(%)         27423
HEX7          27846
color7        27846
비율7           27846
H7(각도)        27846
S7(%)         27846
L7(%)         27846
HEX8          28037
color8        28037
비율8           28037
H8(각도)        28037
S8(%)         28037
L8(%)         28037
HEX9          28129


In [ ]:
!gsutil cp gs://de-project2-bucket-1/kyobo/csv/combined/novel_no_color.csv /content/drive/MyDrive/code/novel_no_color.csv

import pandas as pd
novel_df = pd.read_csv('/content/drive/MyDrive/code/novel_no_color.csv')
novel_df.head()

Copying gs://de-project2-bucket-1/kyobo/csv/combined/novel_no_color.csv...
| [1 files][ 14.7 MiB/ 14.7 MiB]                                                
Operation completed over 1 objects/14.7 MiB.                                     


,index,product_code,product_id,title,list_price,sale_price,discount_rate,point_rate,expected_points,author,...,design_label,title_position,title_direction,detected_objects,text_list,lang_list,text_width_list,text_height_list,text_ratio_list,text_ratio_sum
0,1,9788998441012,S000001632467,모순,"13,000","11,700",10%,5%,650,양귀자,...,typographic,하중,가로,animal,"색, 내 상의 부미논 나무있다, 기사비 칸 안 심음 만한 깊이도 없다, 이자계 신아...","ko, ko, ko, ko","52, 108, 148, 114","84, 16, 14, 16","0.0329, 0.013, 0.0156, 0.0138",0.0753
1,2,9791162203620,S000001808685,파과,"15,800","14,220",10%,5%,790,구병모,...,illustration,중중,알수없음,flower,"구멍보 장편소선, 유지철, 영화 <파과> 원작 소설, 전 세계 13개국 수습, 뉴용...","ko, ko, ko, sw","84, 206, 244, 42","16, 24, 16, 12","0.0103, 0.0381, 0.0301, 0.0039",0.0824
2,3,9788936434120,S000000610612,소년이 온다,"15,000","13,500",10%,5%,750,한강,...,typographic,중중,세로,flower,"#향, $","ko, unknown","65, 30","119, 58","0.0593, 0.0133",0.0726
3,4,9788957753354,S000216345011,포기할 자유,"18,500","16,650",10%,5%,920,이재구,...,photo,하중,가로,landscape,"물, 흙, 피보다이념 이념보다 톤음 쫓는, 핑제들의 비극적인 대서사시!, 가주뭄 비...","ko, ko, ko, ko, ko, ko","44, 14, 162, 150, 216, 32","172, 42, 20, 20, 14, 8","0.0568, 0.0044, 0.0243, 0.0225, 0.0227, 0.0019",0.1326
4,5,9788936439743,S000216111714,혼모노,"18,000","16,200",10%,5%,900,성해나,...,photo,하오,가로,없음,"@모뇌, 지금 가장 성명리고, ""넷플리스 왜 보나, 뜨거오 이I, 성해나 책 보면 ...","ko, ko, ko, ko, ko, ko, ko, ko, ko","68, 70, 128, 44, 170, 42, 44, 38, 244","24, 12, 20, 12, 24, 14, 14, 12, 14","0.0124, 0.0064, 0.0194, 0.004, 0.0309, 0.0045,...",0.1117


In [ ]:
# 필요한 컬럼 리스트
cols_to_merge = ['product_id', 'color1', 'color2', 'color3', '비율1', '비율2', '비율3']
color_subset = color_df[cols_to_merge]

# left join (novel_df 기준)
novel_df = novel_df.merge(color_subset, on='product_id', how='left')
novel_df.head()

,index,product_code,product_id,title,list_price,sale_price,discount_rate,point_rate,expected_points,author,...,text_width_list,text_height_list,text_ratio_list,text_ratio_sum,color1,color2,color3,비율1,비율2,비율3
0,1,9788998441012,S000001632467,모순,"13,000","11,700",10%,5%,650,양귀자,...,"52, 108, 148, 114","84, 16, 14, 16","0.0329, 0.013, 0.0156, 0.0138",0.0753,pastel_yellow,검정색 계열,회색 계열,87.8,10.6,1.1
1,2,9791162203620,S000001808685,파과,"15,800","14,220",10%,5%,790,구병모,...,"84, 206, 244, 42","16, 24, 16, 12","0.0103, 0.0381, 0.0301, 0.0039",0.0824,basic_purple,NaN,NaN,99.7,NaN,NaN
2,3,9788936434120,S000000610612,소년이 온다,"15,000","13,500",10%,5%,750,한강,...,"65, 30","119, 58","0.0593, 0.0133",0.0726,basic_yellow,basic_orange,basic_orange,85.7,13.6,0.2
3,4,9788957753354,S000216345011,포기할 자유,"18,500","16,650",10%,5%,920,이재구,...,"44, 14, 162, 150, 216, 32","172, 42, 20, 20, 14, 8","0.0568, 0.0044, 0.0243, 0.0225, 0.0227, 0.0019",0.1326,흰색 계열,pastel_yellow,NaN,99.5,0.3,NaN
4,5,9788936439743,S000216111714,혼모노,"18,000","16,200",10%,5%,900,성해나,...,"68, 70, 128, 44, 170, 42, 44, 38, 244","24, 12, 20, 12, 24, 14, 14, 12, 14","0.0124, 0.0064, 0.0194, 0.004, 0.0309, 0.0045,...",0.1117,basic_green,basic_red,pastel_orange,79.5,18.0,0.6


In [ ]:
# 필요한 컬럼 리스트
cols_to_merge = ['product_id', 'color4', 'color5', '비율4', '비율5']
color_subset = color_df[cols_to_merge]

# left join (novel_df 기준)
novel_df = novel_df.merge(color_subset, on='product_id', how='left')
novel_df.head()

,index,product_code,product_id,title,list_price,sale_price,discount_rate,point_rate,expected_points,author,...,color1,color2,color3,비율1,비율2,비율3,color4,color5,비율4,비율5
0,1,9788998441012,S000001632467,모순,"13,000","11,700",10%,5%,650,양귀자,...,pastel_yellow,검정색 계열,회색 계열,87.8,10.6,1.1,basic_navy,검정색 계열,0.1,0.1
1,2,9791162203620,S000001808685,파과,"15,800","14,220",10%,5%,790,구병모,...,basic_purple,no_color,no_color,99.7,no_color,no_color,NaN,NaN,NaN,NaN
2,3,9788936434120,S000000610612,소년이 온다,"15,000","13,500",10%,5%,750,한강,...,basic_yellow,basic_orange,basic_orange,85.7,13.6,0.2,NaN,NaN,NaN,NaN
3,4,9788957753354,S000216345011,포기할 자유,"18,500","16,650",10%,5%,920,이재구,...,흰색 계열,pastel_yellow,no_color,99.5,0.3,no_color,NaN,NaN,NaN,NaN
4,5,9788936439743,S000216111714,혼모노,"18,000","16,200",10%,5%,900,성해나,...,basic_green,basic_red,pastel_orange,79.5,18.0,0.6,basic_yellow,basic_orange,0.4,0.2


In [ ]:
novel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28070 entries, 0 to 28069
Data columns (total 43 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   index             28070 non-null  int64  
 1   product_code      28070 non-null  int64  
 2   product_id        28070 non-null  object 
 3   title             28070 non-null  object 
 4   list_price        28070 non-null  object 
 5   sale_price        28070 non-null  object 
 6   discount_rate     28070 non-null  object 
 7   point_rate        28070 non-null  object 
 8   expected_points   28070 non-null  object 
 9   author            28070 non-null  object 
 10  publisher         28070 non-null  object 
 11  pub_date          28070 non-null  object 
 12  category          28070 non-null  object 
 13  image_url         28070 non-null  object 
 14  category_code     28070 non-null  int64  
 15  TopCategory       28070 non-null  object 
 16  MidCategory       28070 non-null  object

In [ ]:
novel_df.isnull().sum()

,0
index,0
product_code,0
product_id,0
title,0
list_price,0
sale_price,0
discount_rate,0
point_rate,0
expected_points,0
author,0


In [ ]:
# 결측치가 존재하는 열 리스트
columns_to_fill = [
    "color4",
    "color5",
    "비율4",
    "비율5"
]

# 각 열의 NaN 값을 "no_color"로 대체
novel_df[columns_to_fill] = novel_df[columns_to_fill].fillna("no_color")

In [ ]:
novel_df.isnull().sum()

,0
index,0
product_code,0
product_id,0
title,0
list_price,0
sale_price,0
discount_rate,0
point_rate,0
expected_points,0
author,0


In [ ]:
# GCS 저장
import os
from google.cloud import storage
import pandas as pd
import tempfile

# 환경변수 설정
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/drive/MyDrive/valid-might-460212-k2-aabca356a049.json'

# GCS 버킷 및 경로 설정
GCS_BUCKET = 'de-project2-bucket-1'
GCS_UPLOAD_PATH = 'kyobo/csv/combined/novel_final_data_MJ_0615.csv'  # 업로드할 경로 및 파일명

# GCS 클라이언트 생성
client = storage.Client()
bucket = client.bucket(GCS_BUCKET)

# novel_df를 임시 파일로 저장한 후 업로드
with tempfile.NamedTemporaryFile(mode='w', suffix='.csv', delete=False) as temp:
    novel_df.to_csv(temp.name, index=False)
    blob = bucket.blob(GCS_UPLOAD_PATH)
    blob.upload_from_filename(temp.name)
    print(f'✅ novel_df successfully uploaded to gs://{GCS_BUCKET}/{GCS_UPLOAD_PATH}')

✅ novel_df successfully uploaded to gs://de-project2-bucket-1/kyobo/csv/combined/novel_final_data_MJ_0615.csv


# 전처리

In [ ]:
print(novel_df)

       index   product_code     product_id                          title  \
0          1  9788998441012  S000001632467                             모순   
1          2  9791162203620  S000001808685                             파과   
2          3  9788936434120  S000000610612                         소년이 온다   
3          4  9788957753354  S000216345011                         포기할 자유   
4          5  9788936439743  S000216111714                            혼모노   
...      ...            ...            ...                            ...   
28065    271  9791192776415  S000201435671  해리 포터 래번클로 기숙사 에디션 5~7 세트(무선)   
28066    275  9791192776262  S000209650564      해리 포터와 아즈카반의 죄수(미나리마 에디션)   
28067    396  9788934972068  S000000597140                       셜록: 크로니클   
28068    450  9788937834356  S000000625402                            동창생   
28069    624  9788956056715  S000000814525                     학교 2013 세트   

      list_price sale_price discount_rate point_rate expected_points  \
0  

In [ ]:
novel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28070 entries, 0 to 28069
Data columns (total 33 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   index             28070 non-null  int64         
 1   product_code      28070 non-null  object        
 2   product_id        28070 non-null  object        
 3   title             28070 non-null  object        
 4   list_price        28070 non-null  object        
 5   sale_price        28070 non-null  object        
 6   discount_rate     28070 non-null  object        
 7   point_rate        28070 non-null  object        
 8   expected_points   28070 non-null  object        
 9   author            28070 non-null  object        
 10  publisher         28070 non-null  object        
 11  pub_date          28070 non-null  datetime64[ns]
 12  category          28070 non-null  object        
 13  image_url         28070 non-null  object        
 14  category_code     2807

In [ ]:
novel_df.isnull().sum()

,0
index,0
product_code,0
product_id,0
title,0
list_price,0
sale_price,0
discount_rate,0
point_rate,0
expected_points,0
author,0


In [ ]:
# 결측치가 존재하는 열 리스트
columns_to_fill = [
    "text_list",
    "lang_list",
    "text_width_list",
    "text_height_list",
    "text_ratio_list"
]

# 각 열의 NaN 값을 "no_image"로 대체
novel_df[columns_to_fill] = novel_df[columns_to_fill].fillna("no_image")

In [ ]:
novel_df.isnull().sum()

,0
index,0
product_code,0
product_id,0
title,0
list_price,0
sale_price,0
discount_rate,0
point_rate,0
expected_points,0
author,0


In [ ]:
# GCS 저장
import os
from google.cloud import storage
import pandas as pd
import tempfile

# 환경변수 설정
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/drive/MyDrive/valid-might-460212-k2-aabca356a049.json'

# GCS 버킷 및 경로 설정
GCS_BUCKET = 'de-project2-bucket-1'
GCS_UPLOAD_PATH = 'kyobo/csv/combined/novel_no_color.csv'  # 업로드할 경로 및 파일명

# GCS 클라이언트 생성
client = storage.Client()
bucket = client.bucket(GCS_BUCKET)

# novel_df를 임시 파일로 저장한 후 업로드
with tempfile.NamedTemporaryFile(mode='w', suffix='.csv', delete=False) as temp:
    novel_df.to_csv(temp.name, index=False)
    blob = bucket.blob(GCS_UPLOAD_PATH)
    blob.upload_from_filename(temp.name)
    print(f'✅ novel_df successfully uploaded to gs://{GCS_BUCKET}/{GCS_UPLOAD_PATH}')

✅ novel_df successfully uploaded to gs://de-project2-bucket-1/kyobo/csv/combined/novel_no_color.csv
